In [3]:
import open3d as o3d
import numpy as np  



In [4]:
views = {
            "top": [0, 1, 0],
            "bottom": [0, -1, 0],
            "left": [1, 0, 0],
            "right": [-1, 0, 0],
            "front": [0, 0, -1],
            "back": [0, 0, 1]
        }

In [6]:
def set_camera_position(visualizer, center, distance, view_name="front"):
    ctr = visualizer.get_view_control()

    # Camera position will be distance units away from the center along the z-axis
    camera_position = center + np.array([0, 0, distance])

    # Set the camera parameters
    ctr.set_lookat(camera_position.tolist())  # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name])  # Set the view direction)  # Camera looking straight down the z-axis
    
    # Adjust zoom if necessary
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])


    # Apply the camera extrinsics
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    extrinsic = { 
        "left": np.array([
    [0, 0, 1, 0],    # Rotate to look along the positive x-axis
    [0, 1, 0, 0],    # No rotation around y
    [-1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "right": np.array([
    [0, 0, -1, 0],   # Rotate to look along the negative x-axis
    [0, 1, 0, 0],    # No rotation around y
    [1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "top": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, 1, 0],    # Rotate to look along the positive z-axis
    [0, -1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "bottom": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, -1, 0],   # Rotate to look along the negative z-axis
    [0, 1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "front": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, 1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ]),
        "back": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, -1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ])
    }
    
    camera_params.extrinsic = extrinsic[view_name]
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary=True)
     # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name]) # Camera looking straight down the z-axis
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])

    visualizer.poll_events()
    visualizer.update_renderer()
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    return camera_params

In [7]:
window_width = 800
window_height = 600
vis = o3d.visualization.Visualizer()
vis.create_window(width=window_width, height=window_height)

mesh_path = "/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg/data/raw/03001627/1a8bbf2994788e2743e99e0cae970928/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())
vis.add_geometry(mesh)

fixed_distance = 1.0
camera_params = set_camera_position(vis, center=np.array([0, 0, 0]), distance=fixed_distance, view_name="front")

# Run the visualizer so you can manually adjust the viewpoint

ctr = vis.get_view_control()
vis.capture_depth_image("depth_image.png")



[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2


In [9]:

depth_image = o3d.io.read_image("depth_image.png")

pcd_from_depth = o3d.geometry.PointCloud.create_from_depth_image(
            depth_image,
            camera_params.intrinsic,
            camera_params.extrinsic,
        )

o3d.visualization.draw_geometries([pcd_from_depth, mesh])    

KeyboardInterrupt: 

: 